In [2]:
from britz_gridworld import GridworldEnv
import numpy as np
import io
import sys
from gym.envs.toy_text import discrete

In [3]:
gridworld = GridworldEnv()

In [27]:
class policy_iteration:
    def __init__(self, environment, discount, threshold):
        self.environment = environment
        self.discount = discount
        self.num_states = environment.nS
        self.num_actions = environment.nA
        self.threshold = threshold
        
    def evaluation(self, policy, maxiter):
        v_0 = np.zeros(self.num_states)

        for i in range(maxiter):
            DELTA = 0
            for i in range(self.num_states):
                v_ = 0
                for act, actprob in enumerate(policy[i]):
                    for prob, state, r, over in self.environment.P[i][act]:
                        v_ += actprob * prob * (r + self.discount * v_0[state])

                DELTA = max(DELTA, np.abs(v_ - v_0[i]))
                v_0[i] = v_

            if DELTA < self.threshold: break

            return np.array(v_0)
        
    def lookahead(self, current_state, V):
        A = np.zeros(self.num_actions)

        for acts in range(self.num_actions):
            for prob, next_state, r, over in self.environment.P[current_state][acts]:
                A[acts] += prob * (r + self.discount * V[next_state])
        return A
    
    def improvement(self, evaluation_function):
        policy = np.ones([self.num_states, self.num_actions]) / self.num_actions
        
        while True:
            V_func = evaluation_function(policy, 10000)
            stable = True
            
            for state in range(self.num_states):
                picked_action = np.argmax(policy[state])
                
                act_vals = self.lookahead(state, V_func)
                optimal_action = np.argmax(act_vals)
                
                if picked_action is not optimal_action:
                    stable = False
                    
                policy[state] = np.eye(self.num_actions)[optimal_action]
                
            if stable:
                return policy, V_func
        

In [28]:
pol_iter = policy_iteration(gridworld, 1.0, 1e-4)
policy, value_function = pol_iter.improvement(pol_iter.evaluation)

KeyboardInterrupt: 